#Mansard Roof Builder

##Examples and reference models

<img src="img/mansardroof.png" width="550px" height="550px">

##Approach

Below are shown the steps used in order to generate the roof:
* from vertices build the shape of the roof bottom
* for each edge of the roof bottom, using the pitch angle, find a plane equation according to the edge and the angle
* find pitches's edge using the intersection of planes
* using the roof height parameter build the pitches
* using the pitches's top vertices build the terrace

##Variables

The following variables are used in order to build the roof:
* roofVertices, a list containing the vertices of the roof bottom shape
> actual parameter: v1 = [0,0]
v2 = [7,0]
v3 = [7,5]
v4 = [6,5]
v5 = [7,7]
v6 = [3,8]
v7 = [0,7]
* roofHeight, the desired height of the roof
* angle, the desired pitch angle

##Geometric and helpers functions

Below are described the geometric and helpers functions used to generate the entire roof structure, for brevity their implementation is omitted, by the way the signatures alongside with docstrings are given

```python
def list2CoupledList(startList):
	"""
	list2CoupledList is a function that given a starting list, return a list containing, for every element in the 
	starting list, a couple (list) made by the original element and its successor, if the original element
	is the last of the original list, the first element of the original list is used as successor.
	E.g. [1,2,3] -> [[1,2],[2,3],[3,1]]
	@param startList: starting list
	@return coupledList: the list containing the couples generated as described above
	"""
   
	#implementation here...
    
def planeFromLine(angle, line):
	"""
	planeFromLine is a function that given a line and an angle, 
    return the 4 coefficients that describe a plane passing
	through the line, the plane contain both the former and the latter point of the line.
	@param angle: the rotation that describe a particular plane
	@param line: the line used to describe the boundle of planes
	@return planesParam: a list containing the 4 coefficients that describe a plane
	"""
    
    #implementation here...
```

##roofBuilder() function

```python
def roofBuilder(verts, angle, height):
	"""
	roofBuilder is a function that given a list of vertices (roof bottom), an angle and an height, 
    return an HPC model of a roof.
	@param verts: the vertices that define the shape of the roof bottom
	@param angle: the angle used to rotate the roof pitches
	@param height: the desired height of the roof
	@return roof: the HPC model of the generated roof
	"""

	lines = list2CoupledList(verts)

	roofBase = SOLIDIFY(POLYLINE(verts + [verts[0]]))

	planes = []
	for line in lines: 
		planes.append(planeFromLine(angle,line))

	#considering planes as couples
	couplePlanes = list2CoupledList(planes)

	roofTop = []
	linesEquations = []

	#calculating lines equations through planes intersections
	for couple in couplePlanes:
		x, y, z = symbols('x y z')
		solved = solve([Eq(couple[0][0]*x+couple[0][1]*y+couple[0][2]*z, couple[0][3]),
			Eq(couple[1][0]*x+couple[1][1]*y+couple[1][2]*z, couple[1][3])])
		linesEquations.append(solved)
		roofTop.append([round(float(solved[x].subs(z,roofHeight)),2), round(float(solved[y].subs(z,roofHeight)),2)])

	roofTop.append(roofTop[0])
	terrace = T([3])([roofHeight])(SOLIDIFY(POLYLINE(roofTop)))

	coupleLines = list2CoupledList(linesEquations)
	roofPitch = []

	#building roof pitches
	for couple in coupleLines:
		base1 = [round(float((couple[0])[x].subs(z,0)),2),round(float((couple[0])[y].subs(z,0)),2),0]
		base2 = [round(float((couple[1])[x].subs(z,0)),2),round(float((couple[1])[y].subs(z,0)),2),0]
		top1 = [round(float((couple[0])[x].subs(z,roofHeight)),2),round(float((couple[0])[y].subs(z,roofHeight)),2),roofHeight]
		top2 = [round(float((couple[1])[x].subs(z,roofHeight)),2),round(float((couple[1])[y].subs(z,roofHeight)),2),roofHeight]
		points = [base1, base2, top2, top1, base1]
		faces = [[1,2,3,4]]
		roofPitch.append(TEXTURE("textures/roof.jpg")(MKPOL([points, faces, 1])))

	roofPitch = STRUCT(roofPitch)

	return STRUCT([TEXTURE("textures/surface.jpg")(terrace), roofBase, roofPitch])
```

##Results

Below are shown two executions of the main functions in such a way that the differences of the pitch angle and of the roof height are clearly visible

#Actual parameter: height roof = 1, pitch = $\pi$/3
<table>
 <tbody>
    <tr>
        <td><img src="img/tetto1.png" style="width: 500px; height: 400px"></td>
        <td><img src="img/tetto3.png" style="width: 500px; height: 400px"></td>
    </tr>
</tbody>
</table>

#Actual parameter: height roof = 2, pitch = $\pi$/6
<table>
 <tbody>
    <tr>
        <td><img src="img/tetto2.png" style="width: 500px; height: 400px"></td>
        <td><img src="img/tetto4.png" style="width: 500px; height: 400px"></td>
    </tr>
</tbody>
</table>